## Miscellaneous

In [3]:
#import packages
import pandas as pd
from selenium import webdriver
from time import sleep
import sys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
#Code for opening a chrome_driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('../../chromedriver.exe',options=chrome_options)

C:\Users\xghan\AppData\Local\Temp\ipykernel_6688\1320990086.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../../chromedriver.exe',options=chrome_options)


In [5]:
driver.close()

## Obtaining HTML Description for Jobstreet Listings

In [6]:
df_jobstreet = pd.read_csv("jobstreet_listings_clean.csv", encoding = "ISO-8859-1")

In [ ]:
df_jobstreet.head()

In [7]:
#Function goes to url to sieve out the description in raw html form
def get_description_html(url):
  #access the webpage
  driver.get(url)
  #wait for job description to appear
  element = WebDriverWait(driver, 10).until(
              EC.presence_of_element_located((By.XPATH,"//div[@data-automation='jobDescription']"))
          )
  #Get the inner HTML of the job description
  description_html  = driver.find_element(By.XPATH,"//div[@data-automation='jobDescription']").get_attribute('innerHTML')
  return description_html

In [ ]:
#Adds the raw html form to the table
result = []
count=0
for i in df_jobstreet['url']:
  try:
    print(count)
    result.append(get_description_html(i))
    count+=1
  except Exception as e:
    print(e)
    result.append("")

df_jobstreet['description_html'] = result

In [13]:
import re
#Function to remove HTML Tags
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, ' ', raw_html)
  return cleantext

In [14]:
#Reobtain the description column with proper spaces. Some words were clumped together previously
result_description =[]
for i in df_jobstreet['description_html']:
  i = cleanhtml(i)
  result_description.append(i)
df_jobstreet['description'] = result_description

In [16]:
#Save file for future use
df_jobstreet.to_csv("jobstreet_listings_withhtml.csv",index=False)

## Cleaning pay_range column for MCF Jobs

In [17]:
df_mcf = pd.read_csv("mcf_processed.csv")

In [ ]:
df_mcf.head()

In [18]:
#Dropping index columns
df_mcf = df_mcf.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [19]:
#Removing html tags from pay_range
df_mcf['pay_range'] = df_mcf['pay_range'].map(lambda x: cleanhtml(x))

In [20]:
#Standardising pay_range format to "$x-$y"
df_mcf['pay_range'] = df_mcf['pay_range'].map(lambda x: x.replace("to","-"))

In [22]:
#Saving file for future use
df_mcf.to_csv("mcf_processed2.csv", index=False)

## Cleaning for Jobsdb

In [24]:
df_jobsdb1 = pd.read_csv("jobsdb-1.csv")
df_jobsdb2 = pd.read_csv("jobsdb-2.csv")

In [29]:
#Merging two parts of the jobsdb file
df_jobsdb = pd.concat([df_jobsdb1,df_jobsdb2]).reset_index()

In [ ]:
df_jobsdb.head()

In [34]:
#Function obtains the pay_range from sub_clean standardizes its format
def get_salary(input_list):
  salary = ""
  try:
    if "per month" in input_list[0]:
      salary = input_list[0]
      salary = salary.replace("per month","")
      return salary
    else:
      salary = None
  except:
    salary = None
    return salary

In [35]:
#Obtain the employment type info from sub_clean
def get_emp_type(input_list):
  emp_type=""
  try:
    if "per month" not in input_list[0]:
      emp_type = input_list[0]
      return emp_type
    elif "per month" not in input_list[1]:
      emp_type = input_list[1]
      return emp_type
    else:
      emp_type = None
      return emp_type
  except:
    emp_type = None
    return emp_type

In [36]:
import ast
#Extracting salary and employment type from the data
df_jobsdb['salary-range'] = df_jobsdb['sub_clean'].map(lambda x: get_salary(ast.literal_eval(x)))
df_jobsdb['emp-type'] = df_jobsdb['sub_clean'].map(lambda x: get_emp_type(ast.literal_eval(x)))

In [37]:
#Extracting company and address from coloc column
df_jobsdb['company'] = df_jobsdb['coloc'].map(lambda x: x.split(',')[0])
df_jobsdb['address'] = df_jobsdb['coloc'].map(lambda x: x.split(',')[1])

In [40]:
#Dropping redundant columns
df_jobsdb = df_jobsdb.drop(columns=["sub_clean",'level_0','Unnamed: 0','coloc','sub_clean','sub_dirty','index'])

In [43]:
#Saved for future references
df_jobsdb.to_csv('jobsdb_cleaned.csv')

## Unifying the Data

In [54]:
#Converting column to title case in MCF
df_mcf['company'] = df_mcf['company'].map(lambda x: x.title())

In [179]:
#Enforcing standardized format ($x-$y) for pay_range in jobstreet
import math
from numpy import NaN
def format_pay(input_str):
    if str(input_str)=='nan':
        return None
    else:
        return str(input_str).replace("SGD??", "$")
df_jobstreet["pay_range"] = df_jobstreet["pay_range"].map(lambda x: format_pay(x))

In [257]:
## Selecting relevant and common columns
## sector is removed from the dataframes as information can often be found within the description
df_mcf = df_mcf[['url','job_title','description','description-clean','pay_range','emp_type','company','address']]
df_jobstreet = df_jobstreet[['url', 'job_title','description_html','description','pay_range','job_type','company','location']]

In [283]:
#Renaming Columns to match df_jobstreet
df_mcf.columns = ['url', 'job_title', 'description_html', 'description', 'pay_range',
       'job_type', 'company', 'location']

In [284]:
#Renaming columns to match df_jobstreet
df_jobsdb.columns=['url', 'job_title', 'description_html', 'description', 'pay_range',
       'job_type', 'company', 'location']

In [695]:
#Combining the 3 tables
df_combined = pd.concat([df_mcf,df_jobsdb,df_jobstreet]).reset_index(drop=True)
df_combined.head()

,url,job_title,description_html,description,pay_range,job_type,company,location
0,https://www.mycareersfuture.gov.sg/job/custome...,PRODUCTION CONTROL MANAGER,<p><strong>JOB DESCIPTION</strong></p>\n<ul>\n...,JOB DESCIPTION\n\n planning and organising pr...,"$2,000 - $3,400","Permanent, Full Time",Snl Logistics Pte Ltd,31 GUL CIRCLE 629569
1,https://www.mycareersfuture.gov.sg/job/enginee...,Design Engineer ( Mechanical / Electrical),<p><strong>SUMMARY</strong></p>\n<ul>\n <li>T...,SUMMARY\n\n This position is responsible...,"$2,500 - $4,500",Full Time,Jamco Aero Design &Amp; Engineering Private Li...,NaN
2,https://www.mycareersfuture.gov.sg/job/sales/b...,Business Development Executive,<p><strong>Job description</strong></p>\n<p>Wh...,Job description\nWho we are:\nWe are a logisti...,"$3,200 - $3,500","Part Time, Permanent",Airpak Express Pte Ltd,"TECHPLAS INDUSTRIAL BUILDING, 45 CHANGI SOUTH ..."
3,https://www.mycareersfuture.gov.sg/job/banking...,Senior / Data Scientist,<p>The ideal candidate should have a good unde...,The ideal candidate should have a good underst...,"$9,000 - $14,000","Permanent, Full Time",Singapore Exchange Limited,"SGX CENTRE I, 2 SHENTON WAY 068804"
4,https://www.mycareersfuture.gov.sg/job/archite...,8890-Sales Consultant [ Digital Software| Saas...,<p><strong>Sales Consultant (Digital Software)...,Sales Consultant (Digital Software)\nLocation:...,"$3,000 - $4,500","Permanent, Full Time",The Supreme Hr Advisory Pte. Ltd.,"SHENTON HOUSE, 3 SHENTON WAY 068805"


In [696]:
df_combined.shape

(11833, 8)

## Preprocessing text data

In [697]:
#Check for missing values
df_combined.isna().sum()

url                    0
job_title              0
description_html       1
description            1
pay_range           7171
job_type             864
company                0
location             598
dtype: int64

In [698]:
#drop empty description columns as it is required to build the recommendation model
df_combined = df_combined.dropna(subset=['description'])

In [ ]:
df_combined.head()

In [699]:
#replace the \n characters with spaces
df_combined['description_tokens'] = df_combined['description'].map(lambda x: x.replace("\n"," "))

In [700]:
#only keep numbers and letters
df_combined['description_tokens'] = df_combined['description_tokens'].map(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [702]:
#Convert to lower case
df_combined['description_tokens'] = df_combined['description_tokens'].map(lambda x: x.lower())

In [703]:
#tokenize words
df_combined['description_tokens'] = df_combined['description_tokens'].map(lambda x: x.split())

In [705]:
#function to remove stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def rm_stopwords(tokens):
    return [i for i in tokens if i not in stop_words and i]

In [706]:
#remove stopwords from tokens
df_combined['description_tokens'] = df_combined['description_tokens'].map(lambda x: rm_stopwords(x))

In [707]:
#stem all words
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def stem_words(tokens):
    return [ps.stem(i) for i in tokens]
df_combined['description_tokens'] = df_combined['description_tokens'].map(lambda x: stem_words(x))

In [710]:
#Create a column that has the full string of title description and company to possibly improve the recommender system by expanding the corpus
df_combined['full_info_tokens'] = df_combined['job_title']+" "+df_combined['description']+" "+df_combined['company']

In [711]:
#preprocess the column the same way as description
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: x.replace("\n"," "))
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: x.lower())
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: x.split())
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: rm_stopwords(x))
df_combined['full_info_tokens'] = df_combined['full_info_tokens'].map(lambda x: stem_words(x))

In [713]:
#Check for duplicates, this sets the first occurence as a non-duplicate (false), and the rest as duplicates (true)
#duplicates are defined as having identical list in the full_info_tokens column
df_combined['duplicated'] = df_combined.duplicated(subset=['full_info_tokens'], keep='first')

In [729]:
#Remove duplicates
df_combined = df_combined[~df_combined['duplicated']]

In [733]:
#Save data
df_combined.to_csv("processed_data.csv", index=False)